In [30]:
using Pkg
Pkg.activate("CT")
using CT
include("global_adder_passthrough.jl")

# benchmark adder_MPO
using ITensors
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO, add1, power_mpo
using Random

L = 12
ancilla = 0
folded = true
seed_vec = 123457
xj = Set([1//3, 2//3])
i1 = 1
p_ctrl = 1.0
p_proj = 0.0
_maxdim = 50
_cutoff = 1e-15
seed = 123457
x0 = nothing
qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
println(phy_list)
println(ram_phy)
println(phy_ram)
rng = MersenneTwister(seed_vec)
rng_vec = seed_vec === nothing ? rng : MersenneTwister(seed_vec)
# initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim);
# println(initial_state)
shift_1_3_bits, shift_1_3_amount = fraction_to_binary_shift(1, 3, L)
# initial_state = productMPS(qubit_site, [1,2,1,2,2,1,2,1,2,1,2,1,1,1,2,2])
initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim);

# initialize haining adder mpo
add1_mpo=MPO(add1(i1,L,phy_ram,phy_list),qubit_site)
add1_6,add1_3=power_mpo(add1_mpo,[div(2^L,6)+1,div(2^L,3)])

# initialize tao adder mpo
carry_links, T_vec, id_vec, gate_vec = initialize_links(L, qubit_site, shift_1_3_bits, ram_phy);
initial_state_1 = copy(initial_state)
initial_state_2 = copy(initial_state)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 12, 2, 11, 3, 10, 4, 9, 5, 8, 6, 7]
[1, 3, 5, 7, 9, 11, 12, 10, 8, 6, 4, 2]


  Activating project at `~/CT_MPS_mini/CT`


MPS
[1] ((dim=2|id=656|"Qubit,Site,n=1"), (dim=50|id=335|"Link,l=1"))
[2] ((dim=50|id=335|"Link,l=1"), (dim=2|id=84|"Qubit,Site,n=2"), (dim=50|id=56|"Link,l=2"))
[3] ((dim=50|id=56|"Link,l=2"), (dim=2|id=873|"Qubit,Site,n=3"), (dim=50|id=563|"Link,l=3"))
[4] ((dim=50|id=563|"Link,l=3"), (dim=2|id=310|"Qubit,Site,n=4"), (dim=50|id=706|"Link,l=4"))
[5] ((dim=50|id=706|"Link,l=4"), (dim=2|id=176|"Qubit,Site,n=5"), (dim=50|id=886|"Link,l=5"))
[6] ((dim=50|id=886|"Link,l=5"), (dim=2|id=287|"Qubit,Site,n=6"), (dim=50|id=601|"Link,l=6"))
[7] ((dim=50|id=601|"Link,l=6"), (dim=2|id=954|"Qubit,Site,n=7"), (dim=32|id=285|"Link,l=7"))
[8] ((dim=32|id=285|"Link,l=7"), (dim=2|id=400|"Qubit,Site,n=8"), (dim=16|id=799|"Link,l=8"))
[9] ((dim=16|id=799|"Link,l=8"), (dim=2|id=22|"Qubit,Site,n=9"), (dim=8|id=588|"Link,l=9"))
[10] ((dim=8|id=588|"Link,l=9"), (dim=2|id=564|"Qubit,Site,n=10"), (dim=4|id=748|"Link,l=10"))
[11] ((dim=4|id=748|"Link,l=10"), (dim=2|id=760|"Qubit,Site,n=11"), (dim=2|id=939|"Link,

In [31]:
using Profile

In [32]:
# ProfileView removed due to glibc compatibility issues
# Use built-in Julia profiling instead:
global_adder(initial_state_2, carry_links, T_vec, id_vec, gate_vec, qubit_site, shift_1_3_bits, ram_phy)


1
  0.074922 seconds (22.51 k allocations: 1.524 MiB, 99.86% compilation time)
2
  0.067304 seconds (22.54 k allocations: 1.523 MiB, 99.85% compilation time)
3
  0.133053 seconds (14.13 k allocations: 1003.570 KiB, 99.93% compilation time)
  0.096818 seconds (14.48 k allocations: 1.006 MiB, 99.88% compilation time)
4
  0.130186 seconds (9.52 k allocations: 670.648 KiB, 99.95% compilation time)
  0.084167 seconds (9.68 k allocations: 680.414 KiB, 99.87% compilation time)
5
  0.000045 seconds (134 allocations: 8.258 KiB)
  0.000018 seconds (135 allocations: 8.789 KiB)
6
  0.000040 seconds (134 allocations: 8.258 KiB)
  0.000018 seconds (135 allocations: 8.789 KiB)
7
  0.000043 seconds (134 allocations: 8.258 KiB)
  0.000013 seconds (135 allocations: 8.789 KiB)
8
  0.000051 seconds (134 allocations: 8.258 KiB)
  0.000056 seconds (135 allocations: 8.789 KiB)
9
  0.000032 seconds (134 allocations: 8.258 KiB)
  0.000019 seconds (135 allocations: 8.789 KiB)
10
  0.000022 seconds (134 allocati

MPS
[1] ((dim=2|id=656|"Qubit,Site,n=1"), (dim=4|id=396|"Link,l=1"))
[2] ((dim=2|id=84|"Qubit,Site,n=2"), (dim=4|id=396|"Link,l=1"), (dim=16|id=905|"Link,l=2"))
[3] ((dim=16|id=905|"Link,l=2"), (dim=2|id=873|"Qubit,Site,n=3"), (dim=32|id=403|"Link,l=3"))
[4] ((dim=32|id=403|"Link,l=3"), (dim=2|id=310|"Qubit,Site,n=4"), (dim=64|id=636|"Link,l=4"))
[5] ((dim=64|id=636|"Link,l=4"), (dim=2|id=176|"Qubit,Site,n=5"), (dim=128|id=927|"Link,l=5"))
[6] ((dim=128|id=927|"Link,l=5"), (dim=2|id=287|"Qubit,Site,n=6"), (dim=256|id=15|"Link,l=6"))
[7] ((dim=256|id=15|"Link,l=6"), (dim=2|id=954|"Qubit,Site,n=7"), (dim=128|id=62|"Link,l=7"))
[8] ((dim=128|id=62|"Link,l=7"), (dim=2|id=400|"Qubit,Site,n=8"), (dim=64|id=456|"Link,l=8"))
[9] ((dim=64|id=456|"Link,l=8"), (dim=2|id=22|"Qubit,Site,n=9"), (dim=32|id=668|"Link,l=9"))
[10] ((dim=32|id=668|"Link,l=9"), (dim=2|id=564|"Qubit,Site,n=10"), (dim=16|id=648|"Link,l=10"))
[11] ((dim=16|id=648|"Link,l=10"), (dim=2|id=760|"Qubit,Site,n=11"), (dim=2|id=395|

In [33]:
Profile.print(format=:flat, sortedby=:count)

# For now, just run the function without profiling:
# global_adder(initial_state_2, carry_links, T_vec, id_vec, gate_vec, qubit_site, shift_1_3_bits, ram_phy)

 Count  Overhead File                    Line Function
 =====  ======== ====                    ==== ========
     1         0 @Base/cartesian.jl        66 macro expansion
     1         0 .../src/XoshiroSimd.jl   248 xoshiro_bulk_simd(rng::TaskLoca...
     1         0 .../src/XoshiroSimd.jl   254 xoshiro_bulk_simd(rng::TaskLoca...
     1         0 @Base/sort.jl           1077 _sort!(v::Vector{ComplexF64}, a...
     1         0 @Base/sort.jl           1046 partition!(t::Vector{ComplexF64...
     1         1 @Base/float.jl             ? isless
     1         0 .../src/XoshiroSimd.jl   253 xoshiro_bulk_simd(rng::TaskLoca...
     1         0 @Base/sort.jl           1098 _sort!(v::Vector{ComplexF64}, a...
     1         1 ...c/set_operations.jl    17 _setdiff(s::NTuple{7, Index{Int...
     1         0 @Base/strings/io.jl      189 string
     1         1 @Base/strings/io.jl      143 print_to_string(::String, ::Var...
     1         1 ...ombiner/contract.jl    14 contract!!(output_tensor::ND